In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Dependencies
import torch as th
import torch.nn.functional as F
from torch import nn

use_cuda = th.cuda.is_available()
th.manual_seed(1)
device = th.device("cuda" if use_cuda else "cpu")

import syft as sy
from syft import workers

hook = sy.TorchHook(th)  # hook torch as always :)

/home/billy/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/billy/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/billy/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/billy/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: Futu

/home/billy/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/billy/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/billy/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/billy/.local/lib/python3.6/site-packages/tensorboard/compat/tensorf

In [3]:
class Net(sy.Plan):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, 20)
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 1)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [4]:
net = Net()

In [5]:
mock_data = th.zeros(1, 2)

In [6]:
net.build(mock_data)

tensor([[0.0210]], grad_fn=<AddmmBackward>)

In [7]:
net.is_built

True

In [8]:
@sy.func2plan(args_shape=[(-1, 1), (-1, 1)])
def loss_fn(target, pred):
    return ((target.view(pred.shape).float() - pred.float()) ** 2).mean()

In [9]:
loss_fn.is_built

True

In [10]:
optimizer = "SGD"

In [11]:
batch_size = 4
optimizer_args = {"lr" : 0.1, "weight_decay" : 0.01}
epochs = 1
max_nr_batches = -1  # not used in this example
shuffle = True

In [12]:
model_config = sy.ModelConfig(model=net,
                              loss_fn=loss_fn,
                              optimizer=optimizer,
                              batch_size=batch_size,
                              optimizer_args=optimizer_args,
                              epochs=epochs,
                              shuffle=shuffle)

## Connect to remote worker

In [13]:
from syft.workers.node_client import NodeClient

In [14]:
alice = NodeClient(hook, "ws://localhost:6666" , id="alice")

In [15]:
from syft.generic.pointers.object_wrapper import ObjectWrapper
from syft.generic.object import AbstractObject

In [16]:
class test_class():
    def __init__(self, a_num, b_num):
        self.a_num = a_num
        self.b_num = b_num

In [17]:
tc = test_class(2, 3)

In [18]:
obj_with_id = ObjectWrapper(id=sy.ID_PROVIDER.pop(), obj=tc)

In [19]:
obj_with_id

<ObjectWrapper id:35201021147 obj:<__main__.test_class object at 0x7fedc8946198>>

In [20]:
ptr = sy.hook.local_worker.send(obj_with_id, alice)

TypeError: can not serialize 'test_class' object

In [15]:
pointer_to_net = net.send(alice)

In [18]:
pointer_to_net

[PointerPlan | me:24096140958 -> alice:88652762960]

In [17]:
pointer_to_net.id

24096140958

In [19]:
pointer_to_net.id_at_location

88652762960

In [15]:
# Send train config
model_config.send(alice)

TypeError: can not serialize 'ModelConfig' object